In [0]:
from pyspark.sql.functions import *

# 1. إعدادات Event Hub
event_hub_namespace = "rg-hospital-eh"
event_hub_name = "rg-hospital-eh"
connection_string = "Endpoint=sb://rg-hospital-eh.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=NsWvR2hMHKIpCfNUffBHwPYMUnwxplUW++AEhJuO/0M="

kafka_options = {
    'kafka.bootstrap.servers': f"{event_hub_namespace}.servicebus.windows.net:9093",
    'subscribe': event_hub_name,
    'kafka.security.protocol': 'SASL_SSL',
    'kafka.sasl.mechanism': 'PLAIN',
    'kafka.sasl.jaas.config': f'kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{connection_string}";',
    'startingOffsets': 'earliest', # تأكد أنها earliest
    'failOnDataLoss': 'false'
}

# 2. قراءة الستريم
raw_df = (spark.readStream
          .format("kafka")
          .options(**kafka_options)
          .load())

json_df = raw_df.selectExpr("CAST(value AS STRING) as raw_json")

# 3. إعدادات التخزين (الحساب الجديد 17)
spark.conf.set(
    "fs.azure.account.key.hospitalstorge17.dfs.core.windows.net",
    "WvLwnc4iFRFgHew6zE9Y55DklpACeOYhA5a3Uz5oCbe16zK0seQxJRs96m5KV4lpDHY56LkqZ3NH+AStfb+C3A=="
)

# 4. تحديد المسارات
# سنضع البيانات في bronze_real
bronze_path = "abfss://bronze@hospitalstorge17.dfs.core.windows.net/bronze_real"
# سنضع الشيك بوينت في مجلد _checkpoints داخل نفس المكان لضمان الوصول
checkpoint_path = "abfss://bronze@hospitalstorge17.dfs.core.windows.net/_checkpoints/patient_flow_v3"

# 5. تنظيف الشيك بوينت القديم (اختياري لكن مفيد لضمان البداية النظيفة)
# dbutils.fs.rm(checkpoint_path, True) 

# 6. الكتابة
query = (json_df
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", checkpoint_path) # مسار جديد كلياً
    .start(bronze_path)
)

# عرض البيانات للتأكد (لا تستخدمها في الإنتاج، فقط للاختبار)
# display(json_df)